# Getting 2018 for Imputation 

#### loading necesary functions - bleh urls broken rn 

In [1]:
# %load ../api_access_snippet.py
#import library
import gspread
# Service client credential from oauth2client
from oauth2client.service_account import ServiceAccountCredentials
# Create scope
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
# create some credential using that scope and content of startup_funding.json
creds = ServiceAccountCredentials.from_json_keyfile_name(
    '../quickstart/g_sheet_creds.json', scope)
# create gspread authorize using that credential
client = gspread.authorize(creds)
my_email = 'matthewjchristy66@gmail.com'


In [2]:
from requests import get
import lxml.html as lh
import pandas as pd


In [ ]:
def read_file(sheet_name):
    out = client.open(sheet_name).sheet1
    out = out.get_all_values()
    out = pd.DataFrame(out, columns=out.pop(0))
    return(out)


In [ ]:
from .data_acq_help_functions import\
    writer, get_url, get_headers, get_table_data,\
    create_df, clean_df


#### make urls to get data 

In [10]:
stat_cat = ['PASSING', 'RUSHING', 'RECEIVING']
pages = [x for x in range(1, 4)]

part_1 = 'http://www.nfl.com/stats/categorystats?tabSeq=0&season=2018&seasonType=REG&d-447263-n=1&d-447263-o=2&statisticCategory='
part_2 = '&conference=null&d-447263-p='
#part_3 = '&d-447263-s=RUSHING_YARDS'

urls = []

for i in stat_cat:
    for j in pages:
        full_url = part_1 + i + part_2 + str(j)  # + part_3
        urls.append(full_url)


In [11]:
passing = pd.DataFrame()
rec = pd.DataFrame()
rush = pd.DataFrame()


In [12]:
for i in range(0, len(urls)):
    source_html = get_url(urls[i])
    colnames = get_headers(source_html)
    table_data = get_table_data(source_html, colnames)
    raw_df = create_df(table_data)

    if 'PASSING' in urls[i]:
        df = clean_df(raw_df, urls[i], select_vars=[
                      'Player', 'Team', 'Yds/G', 'TD', 'Rate'])
        passing = pd.concat([passing, df])

    elif 'RECEIVING' in urls[i]:
        df = clean_df(raw_df, urls[i], select_vars=[
                      'Player', 'Team', 'Yds/G', 'TD', 'FUM'])
        receiving = pd.concat([rec, df])

    else:
        df = clean_df(raw_df, urls[i], select_vars=[
                      'Player', 'Team', 'Yds/G', 'TD', 'FUM'])
        rushing = pd.concat([rush, df])


KeyError: 'Player'

In [10]:
print('Passing', passing.shape)

print('rec', receiving.shape)

print('rush', rushing.shape)


Passing (212, 6)
rec (200, 5)
rush (200, 5)


## Score the Data

In [14]:
i = 0
source_html = get_url(urls[i])
colnames = get_headers(source_html)
table_data = get_table_data(source_html, colnames)
raw_df = create_df(table_data)


In [16]:
urls[0]  # something with the URLs are broken


'http://www.nfl.com/stats/categorystats?tabSeq=0&season=2018&seasonType=REG&d-447263-n=1&d-447263-o=2&statisticCategory=PASSING&conference=null&d-447263-p=1&d-447263-s=RUSHING_YARDS'

In [ ]:
if 'PASSING' in urls[i]:
    df = clean_df(raw_df, urls[i], select_vars=[
                  'Player', 'Team', 'Yds/G', 'TD', 'Rate'])
    passing = pd.concat([passing, df])


In [12]:
rushing[['Yds/G', 'TD', 'FUM']] = rushing[['Yds/G', 'TD', 'FUM']].astype(float)
rushing['2018_ppg'] = (rushing['Yds/G'] * .1) + \
    (rushing['TD'] * 6/16) - (rushing['FUM']/16)


In [ ]:
receiving[['Yds/G', 'TD', 'FUM']
          ] = receiving[['Yds/G', 'TD', 'FUM']].astype(float)
receiving['2018_ppg'] = (receiving['Yds/G'] * .1) + \
    (receiving['TD'] * 6/16) - (receiving['FUM']/16)


## Writing the data 

In [14]:
writer(data=passing, sheet_name='2018_passing_recap', share_email=my_email)


In [15]:
writer(data=rushing, sheet_name='2018_rushing_recap', share_email=my_email)


In [16]:
writer(data=receiving, sheet_name='2018_receiving_recap', share_email=my_email)
